In [ ]:
# Setup and imports  # noqa: ERA001
import json
import sys
from datetime import datetime, timedelta
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent.parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from meta_ally.lib.auth_manager import AuthManager  # noqa: E402
from meta_ally.util.tool_group_manager import ToolGroupManager  # noqa: E402

print("✅ Imports successful!")

# Capture and Anonymize API Data

This notebook captures data from the ACC stage Ally Config API and anonymizes it for safe storage and later mocking.

In [ ]:
# Initialize AuthManager for ACC stage
auth_manager = AuthManager(
    keycloak_url="https://keycloak.prod.iam-services.aws.inform-cloud.io/",
    realm_name="inform-ai",
    client_id="ally-portal-frontend-acc",
    should_open_browser=True
)

print("✅ AuthManager initialized for ACC stage")

✅ AuthManager initialized for ACC stage


In [ ]:
# Initialize ToolGroupManager and load Ally Config tools
tool_manager = ToolGroupManager(auth_manager)

print("Loading Ally Config tools from ACC stage...")
tool_manager.load_ally_config_tools(
    openapi_url="https://ally-config-ui.acc.copilot.aws.inform-cloud.io/openapi.json",
    regenerate_models=True,
    require_human_approval=False
)

print("✅ Tools loaded successfully!")

In [17]:
# Configuration: Specify the endpoint and time range for data capture
# UPDATE THESE VALUES AS NEEDED
ENDPOINT = "/gb80/website/chatbot/prod"  # Change to your endpoint
DAYS_BACK = 30  # Capture last 30 days of data

# Calculate time range
end_time = datetime.now()
start_time = end_time - timedelta(days=DAYS_BACK)

# Format as ISO 8601
start_time_str = start_time.isoformat()
end_time_str = end_time.isoformat()

print("📊 Configuration:")
print(f"  Endpoint: {ENDPOINT}")
print(f"  Start Time: {start_time_str}")
print(f"  End Time: {end_time_str}")
print(f"  Capture Timestamp: {datetime.now().isoformat()}")

📊 Configuration:
  Endpoint: /gb80/website/chatbot/prod
  Start Time: 2025-11-16T11:37:43.877313
  End Time: 2025-12-16T11:37:43.877313
  Capture Timestamp: 2025-12-16T11:37:43.877987


In [ ]:
# Capture data from get_copilot_ratings
print("\n🔄 Fetching copilot ratings...")
ratings_data = await tool_manager.execute_tool_safely(
    "get_copilot_ratings",
    endpoint=ENDPOINT,
    start_time=start_time_str,
    end_time=end_time_str
)

print(f"✅ Captured {len(ratings_data) if ratings_data else 0} rating records")
if ratings_data:
    print(f"Sample: {ratings_data[0] if ratings_data else 'No data'}")

In [19]:
# Capture data from get_copilot_sessions
print("\n🔄 Fetching copilot sessions...")
sessions_data = await tool_manager.execute_tool_safely(
    "get_copilot_sessions",
    endpoint=ENDPOINT,
    start_time=start_time_str,
    end_time=end_time_str
)

print(f"✅ Captured {len(sessions_data) if sessions_data else 0} session records")
if sessions_data:
    print(f"Sample session ID: {sessions_data[0].get('session_id', 'N/A') if sessions_data else 'No data'}")


🔄 Fetching copilot sessions...
🔄 Attempting to call: ally_config_get_copilot_sessions
✅ Success! Result type: <class 'list'>
✅ Captured 389 session records
Sample session ID: a14416c9-840b-49ec-8055-867fb3d5b419
✅ Success! Result type: <class 'list'>
✅ Captured 389 session records
Sample session ID: a14416c9-840b-49ec-8055-867fb3d5b419


In [ ]:
# Capture data from get_copilot_cost_daily (both units)
print("\n🔄 Fetching copilot cost data (tokens)...")
cost_data_tokens = await tool_manager.execute_tool_safely(
    "get_copilot_cost_daily",
    endpoint=ENDPOINT,
    unit="tokens"
)

print("\n🔄 Fetching copilot cost data (euro)...")
cost_data_euro = await tool_manager.execute_tool_safely(
    "get_copilot_cost_daily",
    endpoint=ENDPOINT,
    unit="euro"
)

print("✅ Captured cost data")
print(f"  Tokens: {len(cost_data_tokens) if cost_data_tokens else 0} days")
print(f"  Euro: {len(cost_data_euro) if cost_data_euro else 0} days")

In [21]:
# Store all captured data in a structured format
captured_data = {
    "metadata": {
        "endpoint": ENDPOINT,
        "capture_timestamp": datetime.now().isoformat(),
        "start_time": start_time_str,
        "end_time": end_time_str,
        "days_back": DAYS_BACK
    },
    "ratings": ratings_data,
    "sessions": sessions_data,
    "cost_tokens": cost_data_tokens,
    "cost_euro": cost_data_euro
}

print("\n📦 Data capture complete!")
print("Total data captured:")
print(f"  - Ratings: {len(ratings_data) if ratings_data else 0}")
print(f"  - Sessions: {len(sessions_data) if sessions_data else 0}")
print(f"  - Cost (tokens): {len(cost_data_tokens) if cost_data_tokens else 0} days")
print(f"  - Cost (euro): {len(cost_data_euro) if cost_data_euro else 0} days")


📦 Data capture complete!
Total data captured:
  - Ratings: 0
  - Sessions: 389
  - Cost (tokens): 31 days
  - Cost (euro): 31 days


## Anonymize Data with AI Agent

Use an AI agent to anonymize sensitive information in the captured data.

In [22]:
# Setup anonymization agent with structured data models
from pydantic import BaseModel, Field
from pydantic_ai import Agent

from meta_ally.agents.model_config import ModelConfiguration


# Define structured models matching the actual API response
class Message(BaseModel):
    """A single message in a conversation."""
    role: str = Field(description="Role of the message sender (e.g., 'user', 'assistant', 'system')")
    content: str = Field(description="Anonymized message content with sensitive info removed")
    timestamp: str = Field(description="ISO 8601 timestamp of the message")


class AnonymizedSession(BaseModel):
    """A single anonymized session/conversation."""
    session_id: str = Field(description="Anonymized session identifier (e.g., 'session_1', 'session_2')")
    timestamp: str = Field(description="ISO 8601 timestamp of the session")
    messages: list[Message] = Field(description="List of anonymized messages in the conversation")


# Create model using ModelConfiguration
model = ModelConfiguration(deployment_name="gpt-4.1-nano").create_model()

anonymization_agent = Agent(
    name="Session Anonymization Agent",
    model=model,
    output_type=AnonymizedSession,
    system_prompt="""You are a session data anonymization specialist.

Your task is to anonymize a single conversation session while preserving its structure and usefulness for testing.

Anonymization rules:
1. Replace the session_id with a generic identifier (will be provided in the prompt)
2. Keep the session timestamp intact
3. For each message:
   - Keep role and timestamp intact
   - Sanitize content:
     * Remove personal names → use "**NAME**"
     * Remove email addresses, phone numbers, addresses -> use **email address/phone number/address removed**
     * Keep the general topic and conversation flow intact
     * Maintain technical terms and conversation structure
     * Remove job titles, locations, and any other identifying details unless they are Inform related.

Example:
- "Hi, I'm John from Acme Corp" → "Hi, I'm **NAME** from **COMPANY**"
- "I need help, I am the CTO at Tech Solutions" → "I need help, I am the **POSITION** at **COMPANY**"
- "Hey does, John Mc Doe work here?" → "Hey does, **NAME** work here?"
- "User: I am looking for a Job at Inform. I am a seasoned software engineer at Tech Solutions for 10 years."
" System: We currently offer a Software Developer Role at Inform" →
"User: I am looking for a Job at **COMPANY**. I am a seasoned software engineer at **COMPANY** for 10 years."
"System: We currently offer a Software Developer Role at Inform"

Return the complete anonymized session with all messages processed.
""",
    retries=3,
)

print("✅ Session anonymization agent ready")

✅ Session anonymization agent ready


In [ ]:
# Anonymize sessions individually (only sensitive data needs anonymization)
import asyncio

from openai import APITimeoutError

print("\n🔒 Anonymizing session data...")

anonymized_data = {
    "metadata": captured_data["metadata"],
    "ratings": ratings_data,  # Ratings don't contain PII - keep as-is
    "cost_tokens": cost_data_tokens,  # Cost data is not sensitive
    "cost_euro": cost_data_euro,
    "sessions": []
}

# Process each session individually for better control and structured output
if sessions_data:
    total_sessions = len(sessions_data)
    print(f"  Processing {total_sessions} sessions individually...")

    for idx, session in enumerate(sessions_data, 1):
        # Generate generic session ID
        generic_session_id = f"session_{idx}"

        # Try to anonymize with retry on timeout
        max_retries = 2
        for attempt in range(max_retries):
            try:
                # Anonymize this session
                result = await anonymization_agent.run(
                    f"Anonymize this session (use session ID '{generic_session_id}'): {session}"
                )

                # Add the structured, anonymized session (result.output contains the AnonymizedSession)
                anonymized_data["sessions"].append(result.output.model_dump())
                break  # Success, exit retry loop

            except APITimeoutError:
                if attempt < max_retries - 1:
                    print(f"    ⏱️  Timeout on session {idx}, retrying (attempt {attempt + 2}/{max_retries})...")
                    await asyncio.sleep(2)  # Brief pause before retry
                else:
                    print(f"    ❌ Failed session {idx} after {max_retries} attempts: timeout")
            except Exception as e:
                print(f"    ❌ Error on session {idx}: {str(e)[:100]}")
                break  # Don't retry on other errors

        # Progress indicator
        if idx % 10 == 0 or idx == total_sessions:
            print(f"    ✓ Processed {idx}/{total_sessions} sessions")

    print(f"  ✅ Anonymized {len(anonymized_data['sessions'])} of {total_sessions} sessions")
else:
    print("  ⚠️  No sessions to anonymize")

print("\n✅ Data preparation complete!")

In [ ]:
# Save anonymized data to disk
output_dir = project_root / "Data" / "api_mock_data"
output_dir.mkdir(parents=True, exist_ok=True)

# Create filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"anonymized_api_data_{timestamp}.json"

# Save to file
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(anonymized_data, f, indent=2, default=str)

print(f"\n💾 Data saved to: {output_file}")
print(f"   File size: {output_file.stat().st_size / 1024:.2f} KB")

# Also save as "latest" for easy reference
latest_file = output_dir / "anonymized_api_data_latest.json"
with open(latest_file, "w", encoding="utf-8") as f:
    json.dump(anonymized_data, f, indent=2, default=str)

print(f"   Also saved as: {latest_file}")


💾 Data saved to: /home/jschillb/meta_ally/Data/api_mock_data/anonymized_api_data_20251216_115524.json
   File size: 991.60 KB
   Also saved as: /home/jschillb/meta_ally/Data/api_mock_data/anonymized_api_data_latest.json


In [ ]:
# Print summary
print("\n" + "=" * 80)
print("📊 CAPTURE AND ANONYMIZATION SUMMARY")
print("=" * 80)
print(f"Endpoint: {ENDPOINT}")
print(f"Time Range: {start_time_str} to {end_time_str}")
print(f"Capture Time: {anonymized_data['metadata']['capture_timestamp']}")
print("\nData Captured:")
print(f"  ✓ Ratings: {len(anonymized_data['ratings']) if anonymized_data['ratings'] else 0} records")
print(f"  ✓ Sessions: {len(anonymized_data['sessions']) if anonymized_data['sessions'] else 0} records")
print(f"  ✓ Cost (tokens): {len(anonymized_data['cost_tokens']) if anonymized_data['cost_tokens'] else 0} days")
print(f"  ✓ Cost (euro): {len(anonymized_data['cost_euro']) if anonymized_data['cost_euro'] else 0} days")
print(f"\nOutput: {output_file}")
print("=" * 80)

In [26]:
# Inspect anonymization quality - print random samples
import random

print("\n" + "=" * 80)
print("🔍 ANONYMIZATION QUALITY INSPECTION")
print("=" * 80)

if anonymized_data["sessions"]:
    # Sample up to 20 random sessions
    sample_size = min(20, len(anonymized_data["sessions"]))
    sample_sessions = random.sample(anonymized_data["sessions"], sample_size)

    print(f"\nShowing {sample_size} random anonymized sessions:\n")

    for i, session in enumerate(sample_sessions, 1):
        print(f"\n{'─' * 80}")
        print(f"Sample #{i}")
        print(f"{'─' * 80}")
        print(f"Session ID: {session['session_id']}")
        print(f"Timestamp: {session['timestamp']}")
        print(f"Messages: {len(session['messages'])}")
        print()

        # Print all messages in the session
        for msg_idx, message in enumerate(session['messages'], 1):
            print(f"  Message {msg_idx} [{message['role']}] @ {message['timestamp']}")
            print(f"  Content: {message['content']}")
            print()
else:
    print("\n⚠️  No sessions to inspect")

print("=" * 80)


🔍 ANONYMIZATION QUALITY INSPECTION

Showing 20 random anonymized sessions:


────────────────────────────────────────────────────────────────────────────────
Sample #1
────────────────────────────────────────────────────────────────────────────────
Session ID: session_325
Timestamp: 2025-12-12T07:46:12.581957+00:00
Messages: 5

  Message 1 [assistant] @ 2025-12-12T07:46:12.581957+00:00
  Content: Was möchten Sie über INFORM wissen?

  Message 2 [user] @ 2025-12-12T07:46:12.717547+00:00
  Content: Welche Stellenangebote sind derzeit ausgeschrieben?

  Message 3 [assistant] @ 2025-12-12T07:46:13.774833+00:00
  Content: [Message content unavailable]

  Message 4 [tool] @ 2025-12-12T07:46:44.568608+00:00
  Content: Fehler beim Aufrufen des Backends. Versuche, die Verbindung wiederherzustellen...

  Message 5 [assistant] @ 2025-12-12T07:46:47.202151+00:00
  Content: Derzeit kann ich keine aktuellen Stellenangebote von INFORM abrufen. Wenn Sie möchten, kann ich Ihnen helfen, wie Sie direkt 